In [1]:
import pandas as pd
import pickle
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta
path = '#'

In [4]:
# import preprocessed data
infile = open('final_data_preprocessed.pkl','rb')
import_file = pickle.load(infile)
infile.close()

# balance out dataset
df1 = import_file[import_file['Erfolg'] == 0]
df2 = import_file[import_file['Erfolg'] == 1]
df1 = df1.sample(n=100000)
df2 = df2.sample(n=100000)

# append balanced dataset
import_file_small = df1.append(df2)
import_file_small.to_pickle('balanced_dataset_small_3months.pkl')

In [ ]:
## only keep neccessary features
smaller_features=import_file_small[['UserID','Erstloesung','Erfolg','Schwierigkeit','Wochentag','ist_Schulzeit','MehrfachFalsch','Testposition__FT', 'Testposition__nt', 'Testposition__pruefung',
       'Testposition__training', 'Testposition__version', 'Testposition__vt',
       'Testposition__zt', 'beendet', 'Fehler', 'HA__HA', 'HA__Self', 'HA__nt',
       'HA__vt', 'HA__zt', 'Klassenstufe', 'Jahredabei',
       'Sex__m', 'Sex__w']]

In [2]:
# import all sentences
infile = open(path+'saetze.pkl','rb')
saetze = pickle.load(infile)
infile.close()
saetze = saetze[['satzID','Schwierigkeit','Art','AufgabenID']]

#Art: only GK
options = ['GK'] 
saetze = saetze[saetze['Art'].isin(options)] 

#Aufgaben ID : all except 0 (0 is kompetenztests) 
options = ['0'] 
saetze = saetze[~saetze['AufgabenID'].isin(options)] 

In [ ]:
## create empty df with null values
# rows: all userIDs from preprocessed and balanced dataset
# columns: all satzIDs
userIDs = import_file_small[['UserID']]
userIDs = userIDs.drop_duplicates()
index_col = userIDs['UserID'].tolist()
satzList = saetze["satzID"].tolist()
df = pd.DataFrame(0, index =index_col,columns =satzList)

In [58]:
## import xmlsaetze and xmlsaetze archiv

infile = open(path+'xmlsaetze.pkl','rb')
xmlsaetze = pickle.load(infile)
infile.close()
xmlsaetze = xmlsaetze[['ID','UserID','UebungsID','SatzID','Erfolg','Datum']]

infile = open(path+'xmlsaetze_archiv.pkl','rb')
xmlsaetze_archiv = pickle.load(infile)
infile.close()
xmlsaetze_archiv = xmlsaetze_archiv[['ID','UserID','UebungsID','SatzID','Erfolg','Datum']]

xmlsaetze = xmlsaetze.append(xmlsaetze_archiv)
xmlsaetze_small = xmlsaetze[['UserID','SatzID','Erfolg','Datum']]

# filter satzID: only GK and not kompetenztests
options = saetze['satzID'].tolist()
xmlsaetze_small = xmlsaetze_small[xmlsaetze_small['SatzID'].isin(options)] 

# filter unserID: only User which appear in preprocessed dataset
options = index_col
xmlsaetze_userid = xmlsaetze_small[xmlsaetze_small['UserID'].isin(options)] 


In [59]:
## add column last_login
# not neccessary in production: in production we use t-3months 
# neccessary for training
last_login = xmlsaetze_userid[['UserID','Datum']].groupby(['UserID']).max()
last_login = pd.DataFrame(last_login.reset_index())
last_login = last_login.rename(columns={"Datum": "last_login"})

# merge last login 
xmlsaetze_userid_last_login= xmlsaetze_userid.merge(last_login, on='UserID', how='left')

In [60]:
## iterate through xmlsaetze to finde historical data for each user

for i in range(xmlsaetze_userid_last_login.shape[0]): #iterate trough dataframe row-wise
    user_cell = xmlsaetze_userid_last_login.iloc[i,0]
    satz_cell = xmlsaetze_userid_last_login.iloc[i,1]
    erfolg_cell = xmlsaetze_userid_last_login.iloc[i,2]
    datum_cell = xmlsaetze_userid_last_login.iloc[i,3]
    last_login_cell = xmlsaetze_userid_last_login.iloc[i,4]

    dateOfExercise = datetime.strptime(str(datum_cell), '%Y-%m-%d %H:%M:%S').date()
    dateOfLastLogin = datetime.strptime(str(last_login_cell), '%Y-%m-%d %H:%M:%S').date()
    acceptedDate = dateOfLastLogin + pd.DateOffset(months=-3) # accepted date calculates the date of the last login minus 3 months

    #check if sentence and user is in df
    if satz_cell in df.columns:
        if user_cell in df.index:
            if(dateOfExercise > acceptedDate):
                # set new cell value
                if(erfolg_cell == 1):
                    df.loc[user_cell,satz_cell] = 1
                elif (erfolg_cell ==0):
                    df.loc[user_cell,satz_cell] = -1          

In [62]:
## merge, and save
historical_join = df.reset_index()
historical_join = historical_join.rename(columns={"index": "UserID"})
historical_join.to_pickle('historicalJoin_3months.pkl')

df = pd.merge(smaller_features, historical_join, how='left', on='UserID')
df.to_pickle('smallSampleSet_3months.pkl')

In [4]:
# drop duplicates
df = df.drop(columns=['UserID'])
df = df.drop_duplicates(keep='first')

# save as pickle
df.to_pickle('FINALsmallSampleSet_3months_without_duplicates.pkl')